# Improvements

Author: Frankie Inguanez<br />
Date: 16/01/2023<br /><br />

Improvements on previous implementation.

In [ ]:
class SudokuExecutor:
    def __init__(self, puzzlesFileName: str, trackingFileName:str, statsFileName: str, errorsFileName: str, offset: int, limit: int):
        self.puzzlesFileName=puzzlesFileName
        self.trackingFileName=trackingFileName
        self.statsFileName=statsFileName
        self.errorsFileName=errorsFileName
        self.offset=offset
        self.limit=limit

class SudokuStats:
    def __init__(self):
        self.guesses = 0
        self.backtracks = 0
        self.executionTime = None
        self.unknowns = 0

    def incrementGuesses(self):
        self.guesses += 1

    def incrementBacktracks(self):
        self.backtracks += 1

    def registerExecutionTime(self, executionTime):
        self.executionTime=executionTime

    def setUnknowns(self, zeros:int):
        self.unknowns=zeros

def getFileLineCount(fileName: str):
    """
    Get number of lines in a file.
    Arguments:
        fileName: the name of the file to process.
    """
    import mmap

    lines = 0
    with open(fileName, "r+", encoding="utf-8") as f:
        bf = mmap.mmap(f.fileno(), 0)

        while bf.readline():
            lines += 1

    return lines
    
def saveError(error, errorsFileName: str):
    """
    Saves an error/exception that is raised.
    Arguments:
        error: the Exception that is raised.
        errorsFileName: the file name where the error will be saved.
    """
    try:
        with open(errorsFileName, "a", encoding="utf-8") as ef:
            ef.write("Encountered error:\n{}\n{}\n{}\n\n".format(type(error), error.args, error))
    except Exception as e:
        # Failed to save error to file
        print("Failed to save original error to file due to:\n{}\n{}\n{}\n\n".format(type(e), e.args, e))
        print("Original error:\n{}\n{}\n{}\n\n".format(type(error), error.args, error))
        
def to2DArray(n: str):
    """
    Convert a string to a 2D 9x9 array.
    Arguments:
        n: an 81 digits in string format.
    """
    return [list(map(int, n[i:i+9])) for i in range(0, 81, 9)]

def toStr(puzzle):
    """
    Converts a puzzle to a string.
    Arguments:
        puzzle: a 2 dimensional array representing the 9x9 puzzle. 
    """
    r = ""

    for row in puzzle:
        r += "".join(map(str, row))

    return r

def getColValues(puzzle, col: int):
    """
    Get column values.
    Arguments:
        puzzle: a 2 dimensional array representing the 9x9 puzzle. 
        col: the column number.
    """
    lst = []
    for row in puzzle:
        lst.append(row[col])

    return lst;

def getBoxValues(puzzle, box: int):
    """
    Get box values. Boxes are 3x3 sub-grids enumerates from top left in a raster fashion
    0, 1, 2
    3, 4, 5
    6, 7, 8
    Arguments:
        puzzle: a 2 dimensional array representing the 9x9 puzzle.
        box: the box identification number.
    """
    return [puzzle[x][y] for x in range((box//3)*3,((box//3)*3)+3) for y in range((box%3)*3, ((box%3)*3)+3)]

def checkList(lst: list):
    """
    Checks if a list contains all numbers from 1 to 9.
    Arguments:
        lst: the list of numbers.
    """
    return set(lst) == set(range(1,10))

def isSolved(puzzle):
    """
    Check if a puzzle has been solved.
    Arguments:
        puzzle: a 2 dimensional array representing the 9x9 puzzle.
    """

    # Check rows
    for row in puzzle:
        if not checkList(row):
            return False

    # Check columns
    for i in range(0,9):
        if not checkList(getColValues(puzzle, i)):
            return False;

    # Check box
    for i in range(0,9):
        if not checkList(getBoxValues(puzzle, i)):
            return False;

    return True

def isValid(puzzle, num: int, pos):
    """
    Checks if a number can be added to a specific position
    Arguments:
        puzzle: a 2 dimensional array representing the 9x9 puzzle.
        num: the number to insert.
        pos: the row and column position to place the digit.
    """
    
    # Check the row
    for i in range(len(puzzle[0])):
        if puzzle[pos[0]][i]==num and pos[1] != i:
            return False

    # Check the column
    for i in range(len(puzzle[1])):
        if puzzle[i][pos[1]]==num and pos[0] != i:
            return False

    # Check box
    box_x = pos[1] // 3
    box_y = pos[0] // 3

    for i in range(box_y*3, box_y*3 + 3):
        for j in range(box_x * 3, box_x*3 + 3):
            if puzzle[i][j] == num and (i,j) != pos:
                return False

    return True

In [ ]:
def findRandom(puzzle):
    """
    Finds the next empty cell in a random fashion.
    """
    import random

    for row in random.sample(range(0,9),9):
        for col in random.sample(range(0,9),9):
            if puzzle[row][col] == 0:
                return (row, col)

    return None

def findByRow(puzzle):
    """
    Finds the next empty cell in a raster fashion, row by row.
    Arguments:
        puzzle: a 9x9 sudoku puzzle
    """
    
    for row in range(len(puzzle)):
        for col in range(len(puzzle[0])):
            if puzzle[row][col] == 0:
                return (row, col)
            
    return None

def findByCol(puzzle):
    """
    Finds the next empty cell in a column order.
    Arguments:
        puzzle: a 9x9 sudoku puzzle
    """
    for col in range(0,9):
        for row in range(len(puzzle)):
            if puzzle[row][col]==0:
                return (row, col)
    
    return None

def findByBox(puzzle, mode):
    """
    Finds the next empty cell searching first by box then by row.
    Arguments:
        puzzle: a 9x9 sudoku puzzle
        mode:   4 searches for boxes sequentially, 
                5 searches for boxes in a zig-zag fashion, 
                6 searches for boxes in spiral fashion, 
                7 searches for boxes in a semi zig-zag fashion,
                8 searches for boxes randomly
    """
    import random

    if mode==4:
        boxes=range(0,9)
    elif mode==5:
        boxes=[0,1,2,5,4,3,6,7,8]
    elif mode==6:
        boxes=[0,1,2,5,8,7,6,3,4]
    elif mode==7:
        boxes=[0,1,4,3,6,7,8,5,2]
    elif mode==8:
        boxes=random.sample(range(0,9),9)
    elif mode==9:
        boxes=[0,4,8,1,2,3,5,6,7]

    for box in boxes:
        for row in range((box//3)*3,((box//3)*3)+3):
            for col in range((box%3)*3, ((box%3)*3)+3):
                if puzzle[row][col]==0:
                    return (row, col)

    return None

def findEmpty(puzzle, searchMode):
    """
    Finds the next empty cell in a 9x9 sudoku puzzle.
    Arguments:
        puzzle: the 9x9 sudoku puzzle.
        searchMode: defines how the puzzle is parsed: 
                    1 by row; 
                    2 by col; 
                    3 random;
                    4 by box sequentially; 
                    5 by box in a zig-zag; 
                    6 by box in a spiral; 
                    7 by box in a semi-zig-zag;
                    8 by box randomly;
                    9 by box diagonal;
    """
    if searchMode==1:
        return findByRow(puzzle)
    elif searchMode==2:
        return findByCol(puzzle)
    elif searchMode==3:
        return findRandom(puzzle)
    elif searchMode>=4 and searchMode<=9:
        return findByBox(puzzle, searchMode)
    
    return None

In [ ]:
def getGuesses(puzzle, guessMode):
    """
    Gets numbers to guess.
    Arguments:
        puzzle: the 9x9 sudoku puzzle.
        guessMode: the guessing mode. 1 for sequential, 2 for random.
    """
    import random

    if guessMode==1:
        return range(1,10)
    elif guessMode==2:
        return random.sample(range(1,10),9)

    return None

In [2]:
## Adding tracking of solutions

class SudokuConfig:
    def __init__(self, searchMode: int, guessMode: int, tracking: bool):
        self.searchMode=searchMode
        self.guessMode=guessMode
        self.tracking=tracking

In [3]:
def backtracking(board: list, history: list, stats: SudokuStats, config: SudokuConfig):
    """
    Solves a 9x9 sudoku puzzle using backtracking algorithm.
    Arguments:
        board: the 9x9 puzzle to be solved.
        stats: The statistics object to record algorithm.
        searchMode: defines how the puzzle is parsed: 1 by row; 2 by col; 3 by box sequentially; 4 by box in a zig-zag; 5 by box in a spiral; 6 by box in a semi-zig-zag
        guessMode: defines how numbers are guessed: 1 sequentially; 2 randomly
    """
    # Find the next empty cell
    find = findEmpty(board, config.searchMode)

    # If there is no empty cell than puzzle is complete
    if not find:
        return True
    else:
        row, col = find

    # Get numbers to guess and attempt
    for guess in getGuesses(board, config.guessMode):
        if isValid(board, guess, (row, col)):

            # Brute force guess
            stats.incrementGuesses()
            board[row][col] = guess

            if config.tracking:
                history.append(toStr(board))

            # Attempt to solve rest of puzzle with current choice
            if backtracking(board, history, stats, config):
                return True

            # Invalid puzzle so backtrack
            if (config.tracking):
                history.remove(toStr(board))
            stats.incrementBacktracks()
            board[row][col] = 0

    return False

In [4]:
board = to2DArray('100008509000000000460000000600000000085020047094600302000003090920000700000107005')
board

[[1, 0, 0, 0, 0, 8, 5, 0, 9],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [4, 6, 0, 0, 0, 0, 0, 0, 0],
 [6, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 8, 5, 0, 2, 0, 0, 4, 7],
 [0, 9, 4, 6, 0, 0, 3, 0, 2],
 [0, 0, 0, 0, 0, 3, 0, 9, 0],
 [9, 2, 0, 0, 0, 0, 7, 0, 0],
 [0, 0, 0, 1, 0, 7, 0, 0, 5]]

In [5]:
import timeit

history = []
stats = SudokuStats()
config = SudokuConfig(searchMode=1, guessMode=1, tracking=True)

timeit.timeit(lambda:backtracking(board, history, stats, config), number=1000)

14.403690400002233

In [6]:
board

[[1, 3, 7, 4, 6, 8, 5, 2, 9],
 [2, 5, 8, 3, 1, 9, 4, 7, 6],
 [4, 6, 9, 5, 7, 2, 1, 8, 3],
 [6, 1, 2, 7, 3, 4, 9, 5, 8],
 [3, 8, 5, 9, 2, 1, 6, 4, 7],
 [7, 9, 4, 6, 8, 5, 3, 1, 2],
 [5, 7, 6, 2, 4, 3, 8, 9, 1],
 [9, 2, 1, 8, 5, 6, 7, 3, 4],
 [8, 4, 3, 1, 9, 7, 2, 6, 5]]

In [7]:
history

['130008509000000000460000000600000000085020047094600302000003090920000700000107005',
 '137008509000000000460000000600000000085020047094600302000003090920000700000107005',
 '137408509000000000460000000600000000085020047094600302000003090920000700000107005',
 '137468509000000000460000000600000000085020047094600302000003090920000700000107005',
 '137468529000000000460000000600000000085020047094600302000003090920000700000107005',
 '137468529200000000460000000600000000085020047094600302000003090920000700000107005',
 '137468529250000000460000000600000000085020047094600302000003090920000700000107005',
 '137468529258000000460000000600000000085020047094600302000003090920000700000107005',
 '137468529258300000460000000600000000085020047094600302000003090920000700000107005',
 '137468529258310000460000000600000000085020047094600302000003090920000700000107005',
 '137468529258319000460000000600000000085020047094600302000003090920000700000107005',
 '1374685292583194004600000006000000000850200470946003

In [8]:
for i in range(len(history)):
    print(history[i])
    break

130008509000000000460000000600000000085020047094600302000003090920000700000107005


In [9]:
def generatePuzzle(count: int, zeros: int):
    """
    Generates a 9x9 sudoku puzzle grid.
    Arguments:
        count: the number of puzzles to generate.
        zeros: the number of zeros to inject in the puzzles.
    """
    import random
    
    puzzles = []
    for i in range(count):
        puzzle = '000000000000000000000000000000000000000000000000000000000000000000000000000000000'

        board = to2DArray(puzzle)
        stats = SudokuStats()
        stats.setUnknowns(puzzle.count('0'))
        config = SudokuConfig(searchMode=9, guessMode=2, tracking=False)
        backtracking(board, None, stats, config)

        # Remove digits
        changed = 0
        while changed < zeros:
            row = random.sample(range(0,9),1)[0]
            col = random.sample(range(0,9),1)[0]
            
            if board[row][col]==0:
                continue

            board[row][col]=0
            changed+=1
            
        puzzles.append(toStr(board))

    return puzzles
    

In [10]:
generatePuzzle(2,54)

['001000480000000001006400000040300000018004300520008704967140800000560003000000900',
 '500687020900302400280000005000100002000800700000020004008200007000930006006700800']

In [12]:
import tqdm

# Generate balanced dataset of 50 puzzles for different number of missing digits
with open("balanced_24_45_zeros.txt", "w", encoding="utf-8") as f:
    for i in tqdm.tqdm(range(24,46)):
        puzzles = generatePuzzle(50,i)

        for j in range(len(puzzles)):
            f.write("{}\n".format(toStr(puzzles[j])))


100%|██████████| 22/22 [07:26<00:00, 20.29s/it]
